In [1]:
import glob2
import os.path
import os
from subprocess import PIPE, Popen

In [2]:
def cmdline(command):
    '''
    command: like a linux command 'ls',  'pwd'
    note: there is b' at the begining and
                   \n' at the end 
    '''
    process = Popen(
        args=command,
        stdout=PIPE,
        shell=True
    )
    return process.communicate()[0]

def get_unique(list1):
  
    # initialize a null list
    unique_list = []
  
    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    
    return(unique_list)


def count_file(grid, time, var_lst):
    '''
    grid: there are two grid, '11' and '44'
    time: 'mon' or 'day'
    count total files of each variable in FRONTIER data directory
    /glade/campaign/mmm/c3we/FRONTIER/data/
    '''
    dir_i = '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-' + grid +'/'
    print(dir_i)
    #dir_day = glob2.glob(dir_i + '*/*/*/day')
    #dir_monthly = glob2.glob(dir_i + '*/*/*/mon/')
    cmd = 'ls '
    for var in var_lst:
         aa = str(cmdline(cmd + dir_i + '*/*/*/'+ time +'/' + var + '/*nc |wc -l '))
         aa = aa.replace('b\'', '')   # "b'1517\\n'"
         aa = aa.replace('\\n\'', '')  
         print(aa, var)    
        
def id_list(d1_lst, d0_lst, var_s, var_s1, day_mon):
    '''
    d1_lst: directory
    d0_lst: previous directory
    var_s1: varialbe names from d0_lst, normoally I just pick the variable with most files
    day_mon: 'day' or 'mon', dayly or monthly data
    '''
    id_s = []
    id_all = []
 
    for d1, d0 in zip(d1_lst, d0_lst):
        flnm6_all = glob2.glob(d1 + '/' + var_s1 + '/*')
        #flnm6_all = glob2.glob(d1 + '/zg500/*')
        #print(flnm6_all )
        for flnm6 in flnm6_all:
            # use partition to  get a string after a specific substring?
            #print(flnm6)
         
            yyyymm_s = flnm6.partition(day_mon+"_")[2][:6]
            #print(yyyymm_s)
            # get RCM, 5th '_'
            rcm_s = flnm6.split('_')[5]
        
            # get GCM 3rd '_'
            gcm_s = flnm6.split('_')[2]

            # get scenariao 
            scen_s = flnm6.split('_')[3]

            # get realization
            real_s = flnm6.split('_')[4]

            identifier = flnm6.split('_')[2] + "_" + flnm6.split('_')[3] + "_" + flnm6.split('_')[4] + "_" + flnm6.split('_')[5]
            #print('1',flnm6, yyyymm_s , " " + rcm_s + " " + gcm_s + " " + scen_s  + " " + real_s )
        
            # for each 6hr ua850 dataset, check availability of other variables
            for va in range(len(var_s)):
                flnm = glob2.glob(d0 + '/' + day_mon +'/' + var_s[va] + '/' + var_s[va] + '_*' + gcm_s + '_*' + rcm_s + '_v*' + yyyymm_s + '*nc')
                #print('2', d0 + '/'+day_mon+'/' + var_s[va] + '/' + var_s[va] + '_*' + gcm_s + '_*' + rcm_s + '_v*' + yyyymm_s + '*nc' )
             
                if len(flnm)==0:
                    #if not os.path.exists(flnm[0]):
                    # throw out datasets without all 7 required variables
                    break
                else:
                    if va==len(var_s)-1:
                        yyyymm_id = str(yyyymm_s) + "_" + identifier
                        id_s.append(identifier)
                        id_all.append(yyyymm_id)
                        #print(identifier)
                        
    return id_s, id_all

### Monthly variables Overview
#### count total files of each variable

In [3]:
var_monthly = [ 'zg850','zg500','zg200','ua850','ua500','ua200','va850','va500','va200','ta850','ta500','ta200','hus850'] 

In [4]:
### Grid .11
grid = '11'
day_mon = 'mon'
count_file(grid, day_mon, var_monthly)

/glade/campaign/mmm/c3we/FRONTIER/data/EUR-11/
10 zg850
1517 zg500
1486 zg200
1481 ua850
1486 ua500
1486 ua200
1481 va850
1486 va500
1486 va200
1492 ta850
1497 ta500
1497 ta200
1482 hus850


In [5]:
grid = '44'
count_file(grid, day_mon, var_monthly)

/glade/campaign/mmm/c3we/FRONTIER/data/EUR-44/
0 zg850
534 zg500
503 zg200
503 ua850
503 ua500


ls: cannot access /glade/campaign/mmm/c3we/FRONTIER/data/EUR-44/*/*/*/mon/zg850/*nc: No such file or directory


503 ua200
503 va850
503 va500
503 va200
503 ta850
507 ta500
507 ta200
543 hus850


#### because there are only 10 zg850 files in EUR-11, 
I extract it from variable list. Those 10 files are:

In [6]:
grid = '11'
cmd = 'ls '

# there are 2 data directories EUR-11 and EUR-44
dir_i = '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-' + grid +'/'
dir_day = glob2.glob(dir_i + '*/*/*/day')
dir_monthly = glob2.glob(dir_i + '*/*/*/mon')

flnm_s = str(cmdline(cmd + dir_i + '*/*/*/mon/' + 'zg850' + '/*nc '))
# remove b' at the start of the str
flnm_s = flnm_s.replace('b\'', '')  

# remove n' at the end of str
flnm_s  = flnm_s.replace('n\'', ' ')   

# replace \n with space
flnm_s  = flnm_s.replace('\\n', ' ') 

## remove \ at the end of str
flnm_s  = flnm_s.replace('\\ ', '') 
 
flnm_lst =flnm_s.split(' ')
print('there are ', len(flnm_lst), ' zg850 files and they are')
flnm_lst

there are  10  zg850 files and they are


['/glade/campaign/mmm/c3we/FRONTIER/data/EUR-11/rcp85/CNRM-CERFACS-CNRM-CM5/r1i1p1/mon/zg850/zg850_EUR-11_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_mon_200601-201012.nc',
 '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-11/rcp85/CNRM-CERFACS-CNRM-CM5/r1i1p1/mon/zg850/zg850_EUR-11_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_mon_201101-202012.nc',
 '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-11/rcp85/CNRM-CERFACS-CNRM-CM5/r1i1p1/mon/zg850/zg850_EUR-11_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_mon_202101-203012.nc',
 '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-11/rcp85/CNRM-CERFACS-CNRM-CM5/r1i1p1/mon/zg850/zg850_EUR-11_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_mon_203101-204012.nc',
 '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-11/rcp85/CNRM-CERFACS-CNRM-CM5/r1i1p1/mon/zg850/zg850_EUR-11_CNRM-CERFACS-CNRM-CM5_rcp85_r1i1p1_CLMcom-ETH-COSMO-crCLIM-v1-1_v1_mon_204101-205012.nc',
 '/glade/campaign/mm

#### count those situations with all variables
#####  'zg500','zg200','ua850','ua500','ua200','va850','va500','va200','ta850','ta500','ta200','hus850'

In [7]:
var_monthly = [ 'zg500','zg200','ua850','ua500','ua200','va850','va500','va200','ta850','ta500','ta200','hus850'] 
 
var_s = var_monthly
n_var = len(var_s)
n_var

scenariao = ['evaluation', 'historical',  'rcp26', 'rcp45', 'rcp85']
n_scen = len(scenariao)

In [8]:
# there are 2 data directories EUR-11 and EUR-44

grid = '11'

dir_i = '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-' + grid +'/'
dir_day = glob2.glob(dir_i + '*/*/*/day')
dir_monthly = glob2.glob(dir_i + '*/*/*/mon')

In [9]:
#iterate through day/month lists

d_monthly = []
monthly_dir = []
for mon in dir_monthly:
    d_monthly.append(mon)
    monthly_dir.append(mon[:-4])

In [10]:
#print(d_monthly)
#monthly_dir 

id_s, id_all = id_list(d_monthly, monthly_dir, var_monthly, 'zg500', 'mon')
 

In [11]:
is_all = False

if is_all == True:
    with open('allVar_' + grid + '.txt', 'w') as f:
        for line in get_unique(id_all):
            f.write(f"{line}\n")
else:
    with open('uniqueVar_'+ grid + '.txt', 'w') as f:
        for line in get_unique(id_s):
            f.write(f"{line}\n")
            

In [12]:
cmd = 'grep ' 
flnm = ' uniqueVar_' + grid + '.txt'

for scen in scenariao:
    print(scen)
    aa = str(cmdline(cmd + scen + flnm))
    aa = aa.replace('b\'', '') 
    
    flnm_s=aa.split('\\n')
    # last one is ","
    print(len(flnm_s) - 1)
         

evaluation
9
historical
67
rcp26
28
rcp45
23
rcp85
49


### Daily variables overview
#### count total files for each variable

In [13]:
var_daily = ['rlut', 'pr', 'psl', 'prw', 'clt', 'tas', 'tasmin', 'tasmax']
var_s = var_daily
n_var = len(var_s)
n_var

8

In [19]:
### Grid .11
grid = '44'
day_mon = 'day'
count_file(grid, day_mon, var_daily)

/glade/campaign/mmm/c3we/FRONTIER/data/EUR-44/
1141 rlut
1296 pr
1177 psl
955 prw
981 clt
1308 tas
1212 tasmin
1240 tasmax


In [20]:
### Grid .44
grid = '44' 
count_file(grid, day_mon, var_daily)

/glade/campaign/mmm/c3we/FRONTIER/data/EUR-44/
1141 rlut
1296 pr
1177 psl
955 prw
981 clt
1308 tas
1212 tasmin
1240 tasmax


#### count those situations with all daily variables
'rlut', 'pr', 'psl', 'prw', 'clt', 'tas', 'tasmin', 'tasmax'

In [21]:
# there are 2 data directories EUR-11 and EUR-44
grid = '11'

dir_i = '/glade/campaign/mmm/c3we/FRONTIER/data/EUR-' + grid +'/'
dir_day = glob2.glob(dir_i + '*/*/*/day')

#iterate through day/month lists
d_day = []
day_dir = []
for day in dir_day:
    d_day.append(day)
    day_dir.append(day[:-4]) 
    
id_s, id_all = id_list(d_day, day_dir, var_daily, 'tas', 'day')


In [22]:
if is_all == True:
    with open('allVar_day_' + grid + '.txt', 'w') as f:
        for line in get_unique(id_all):
            f.write(f"{line}\n")
else:
    flnm = 'uniqueVar_day_'+ grid + '.txt'
    with open(flnm, 'w') as f:
        for line in get_unique(id_s):
            f.write(f"{line}\n")

In [23]:
cmd = 'grep '
print(flnm)
for scen in scenariao:
    print(scen)
    aa = str(cmdline(cmd + scen + " " + flnm))
    aa = aa.replace('b\'', '') 
    
    flnm_s=aa.split('\\n')
    # last one is ","
    print(len(flnm_s) - 1)

uniqueVar_day_11.txt
evaluation
7
historical
6
rcp26
17
rcp45
18
rcp85
13
